In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import pandas as pd 

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


/tmp/ipykernel_1723/2851567373.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# df = 
# shape = df.shape
# print(shape)
# print(df.head())

# training, test = torch.utils.data.random_split(df, [int(shape[0]*0.8), int(shape[0]*0.2)])
training = pd.read_csv("data/ParisHousingClass.csv", sep=",", header="infer")
test = pd.read_csv("data/ParisHousingClass_Test.csv", sep=",", header="infer")
features = training.shape[1] - 1

print(f"Training set size: {len(training)}")
print(f"Eval set size: {len(test)}")

categories = training['category'].unique()
category_mapping = {cat: i for i, cat in enumerate(categories)}


Training set size: 8000
Eval set size: 2000


In [4]:
class ParisCategoryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(features, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [37]:
model = ParisCategoryClassifier().to(device)
print(model)

class ParisDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def __len__(self):
        print(self.df.shape)
        return self.df.shape[0]

    def __getitem__(self, idx):
        data = self.df.drop(columns=['category']).iloc[idx]
        label = self.df.iloc[idx]['category']
        # print(f"Label {label}")
        label = category_mapping[label]
        # print(f"Label after mapping {label}")
        # print(f"Data values {data.values}")

        data_tensor = torch.tensor(data.values, dtype=torch.float32)
        # print(f"Data tensor {data_tensor}")
        label_tensor = torch.tensor(label, dtype=torch.int64)
        # print(f"Label tensor {label_tensor}")
        
        return data_tensor, label_tensor

training_set = ParisDataset(training)
test_set = ParisDataset(test)

training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False)

print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(test_set)))



ParisCategoryClassifier(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=17, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)
(8000, 18)
(8000, 18)
(8000, 18)
Training set has 8000 instances
(2000, 18)
Validation set has 2000 instances


In [40]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        # print(f"{outputs.shape} {labels.shape}")
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

train_one_epoch(0, None)



(8000, 18)
(8000, 18)
  batch 1000 loss: nan
(8000, 18)


AttributeError: 'NoneType' object has no attribute 'add_scalar'